# Solving NLP (Natural Language Processing) Problems with HuggingFace Library on AWS for Turkish Language

## Use Case
In this demo, we will demonstrate two NLP use cases.
- _Sentiment Analysis:_ You can find positive, negative and neutral mentions about your business, competitors or any topic provided as text to the Machine Learning model.
- _Question Answering:_ Question-Answering Models are  deep learning models that can answer questions given some context, and sometimes without any context (e.g. open-domain QA). They can extract answer phrases from paragraphs, paraphrase the answer generatively, or choose one option out of a list of given options, and so on.

## Dataset
We will use following datasets:
- _Dataset Card for Turkish Product Reviews:_ This Turkish Product Reviews Dataset contains 235.165 product reviews collected online. There are 220.284 positive, 14881 negative reviews.
- _Turkish NLP Q&A Dataset:_ This dataset is the Turkish Question & Answer dataset on Turkish Science History.

## Approach
Instead of creating a new Machine Learning (ML) model for every new task, we can leverage the concept of *Transfer Learning*.
In particular, we can use generic language models and teach it new tasks by fine-tuning them using corresponding datasets.
In this notebook we will use a Turkish language model created by the MDZ Digital Library team (dbmdz) at the Bavarian State Library (https://github.com/stefan-it/turkish-bert). We will use the Hugging Face Model Hub to download the model (https://huggingface.co/dbmdz/bert-base-turkish-uncased) and then fine-tune it to two  different tasks. We will deploy to SageMaker for real-time inferencing.
- Sentiment Analysis: We will see how the fine tuned model achieves SoTA (State of the Art) performance for Sentiment Analysis for Turkish easily.
- Question Answering:

## How to Run this Notebook in Amazon SageMaker
You can run this notebook in SageMaker Studio. Please select the `PyTorch 1.6 Python 3.6 CPU Optimized` kernel.

## SageMaker Setup

In [ ]:
!pip install transformers -q -U

In [ ]:
!pip install datasets -q -U

In [ ]:
!pip install ipywidgets IProgress -q

In [ ]:
!pip install sagemaker -q -U

In [68]:
!mkdir data

In [69]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20211005T160629
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


#### Define model name

In [70]:
model_name = 'dbmdz/bert-base-turkish-uncased'

## Sentiment Analysis

### Downloading dataset and splitting into test and training sets

We will downlaod the data directly from Huggingface: https://huggingface.co/datasets/turkish_product_reviews

In [71]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer
from sagemaker.huggingface.model import HuggingFacePredictor

In [72]:
dataset_name = 'turkish_product_reviews'
dataset = load_dataset(dataset_name)

Using custom data configuration default
Reusing dataset turkish_product_reviews (/root/.cache/huggingface/datasets/turkish_product_reviews/default/1.0.0/9cc21a14e05e4117ea24b5b916effe55cbc88278441c21e03d3807c9bda2219d)


  0%|          | 0/1 [00:00<?, ?it/s]

We will only take 10% of the data to reduce training time

In [73]:
sample = dataset['train'].train_test_split(test_size=0.1)

Now we split the data into training set (90%) and test set (10%)

In [74]:
dataset = sample['test']
train_test = dataset.train_test_split(test_size=0.1)

In [75]:
train_dataset = train_test['train']
test_dataset = train_test['test']

Now we can inspect the training data

In [76]:
df_train = pd.DataFrame(train_dataset)

In [77]:
pd.set_option('display.max_colwidth', 0)

In [83]:
df_inspect = pd.concat([df_train[df_train['sentiment']==0].head(3), df_train[df_train['sentiment']==1].head(3)])

In [84]:
df_inspect

,sentence,sentiment
29,çok kullanışlı değil benim hoşuma gitmedi.,0
54,almanın bi anlamı yok hiç te anlatıldığı gibi değil ne ses çıkarıyor nede dikkat cekiyor,0
68,tasarım güzel ama arka kamera içeride olsa ve küçük olsa daha iyi olur kamera büyük ve dışarda çabuk zarar gördü kamerası kılıf kullanmama rağmen arka kapak çatladı cep hizasından düştü,0
0,gayet i̇yi̇,1
1,telefonu eşime doğum günü hediyesi olarak aldım ve hediyesini aldığı günden beri elinden düşürmüyor hakikaten çok kullanışlı ve hızlı bir telefon eğer bu telefonu almak isteyip de ikilem de kalanlar var ise hiç düşünmeden mutlaka alsınlar derim. çünkü her türlü özelliğe sahip.,1
2,bence başarili bi̇r satici ürün 2. gün eli̇me ulaşti kurulumu çok basi̇t eli̇ni̇ze emeği̇ni̇ze sağlik,1


Before we can start the training we need to tokenize the data save it in S3

In [85]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['sentence'], padding='max_length', truncation=True)

In [86]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [87]:
train_dataset =  train_dataset.rename_column("sentiment", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("sentiment", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [88]:
s3_prefix_sentiment = 'datasets/turkish_product_reviews'

In [89]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix_sentiment}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix_sentiment}/test'
test_dataset.save_to_disk(test_input_path,fs=s3)

### Model Training

In [90]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters_sentiment={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name': model_name
                 }

In [91]:
huggingface_estimator_sentiment = HuggingFace(entry_point='train.py',
                                    source_dir='./scripts',
                                    instance_type='ml.p3.2xlarge',
                                    instance_count=1,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
                                    hyperparameters=hyperparameters_sentiment,
                                    )

In [92]:
huggingface_estimator_sentiment.fit({'train': training_input_path, 'test': test_input_path}, wait=False)

### Model Deployment

In [ ]:
predictor_sentiment = huggingface_estimator_sentiment.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    wait=False,
    endpoint_name="turkish-sentiment-endpoint"
)

### Model Testing

In [94]:
# This is only required to create a predictor from an already deployed model
predictor_sentiment = HuggingFacePredictor('turkish-sentiment-endpoint')

In [95]:
# Input text: "This is a pretty bad product, I wouldn't recommend this to anyone"
sentiment_input= {"inputs": "Bu oldukça kötü bir ürün, bunu kimseye tavsiye etmem"}
predictor_sentiment.predict(sentiment_input)

[{'label': 'LABEL_0', 'score': 0.9701817631721497}]

In [96]:
#Input text: "I love this shampoo, it makes my hair so shiny"
sentiment_input= {"inputs": "Bu şampuanı seviyorum, saçlarımı çok parlak yapıyor"}
predictor_sentiment.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9980107545852661}]

## Question Answering

### Downloading the data

Taken from https://github.com/TQuad/turkish-nlp-qa-dataset

In [97]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json -q

In [98]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json -q

In [99]:
!mv train-v0.1.json data/train-v0.1.json
!mv dev-v0.1.json data/dev-v0.1.json

The JSON files must be converted so that they can be used in a Q&A model

In [100]:
import json
from datasets import load_dataset

def convert_json(input_filename, output_filename):
    with open(input_filename) as f:
        dataset = json.load(f)

    with open(output_filename, "w") as f:
        for article in dataset["data"]:
            title = article["title"]
            for paragraph in article["paragraphs"]:
                context = paragraph["context"]
                answers = {}
                for qa in paragraph["qas"]:
                    question = qa["question"]
                    idx = qa["id"]
                    answers["text"] = [a["text"] for a in qa["answers"]]
                    answers["answer_start"] = [int(a["answer_start"]) for a in qa["answers"]]
                    f.write(
                        json.dumps(
                            {
                                "id": idx,
                                "title": title,
                                "context": context,
                                "question": question,
                                "answers": answers,
                            }
                        )
                    )
                    f.write("\n")

In [101]:
convert_json('data/train-v0.1.json', 'data/train.json')
convert_json('data/dev-v0.1.json', 'data/val.json')

In [102]:
data_files = {}
data_files["train"] = 'data/train.json'
data_files["validation"] = 'data/val.json'

In [103]:
from datasets import load_dataset
ds = load_dataset("json", data_files=data_files)

Using custom data configuration default-45cc421ab74a1da8


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-45cc421ab74a1da8/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [104]:
df = pd.DataFrame(ds['train'])

In [105]:
df.iloc[7518:7521]

,id,title,context,question,answers
7518,8348,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi'nin kurulma amacı nedir?,"{'text': [' Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması '], 'answer_start': [75]}"
7519,8349,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevinin ne gibi amaçları vardır?,"{'text': ['Gerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak'], 'answer_start': [661]}"
7520,8350,İstanbul Üniversitesi Gözlemevi,"İstanbul Üniversitesi Gözlemevi Araştırma ve Uygulama Merkezi, 1933 yılında Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla açılmıştır. Gözlemsel astronomi konusunda çalışmalara evsahipliği yapan merkez;\r\n\r\nYıldız, güneş, uydu, astroid, kuyrukluyıldız, meteor, metorit ve tutulma gözlemleri yapmak ve gözlem verilerini değerlendirmek. \r\nDünyanın sayılı 200 gözlem veri merkezi ile 1939 yılından bu yana sürdürülen veri alışverişini sürdürmek. \r\nNASA, ESA gibi kuruluşların atmosfer dışından gözlem yapmak amacıyla uzaya gönderdiği yapay uydu verilerini alıp indirgemek ve değerlendirmek\r\nGerek yurt dışından alınan gerekse İstanbul Üniversitesi Gözlemevi'nde elde edilen verileri kullanarak ve gerekli bilgisayar programlarını yazarak araştırmalar yapıp yayınlamak. gibi amaçlar gütmektedir.",İstanbul Üniversitesi Gözlemevi hangi amaçla açılmıştır?,"{'text': ['Fen Fakültesi bünyesinde kurulan Astronomi ve Uzay Bilimleri Bölümü'nün araştırma ve gözlemlerinde kullanılması amacıyla'], 'answer_start': [76]}"


Uploading to S3

In [106]:
s3_prefix_qa = 'datasets/turkish_qa'

In [107]:
!aws s3 cp data/train.json s3://$sagemaker_session_bucket/$s3_prefix/train.json
!aws s3 cp data/val.json s3://$sagemaker_session_bucket/$s3_prefix/val.json

upload failed: data/train.json to s3:////train.json Parameter validation failed:
Invalid bucket name "": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"
upload failed: data/val.json to s3:////val.json Parameter validation failed:
Invalid bucket name "": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"


### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

hyperparameters_qa={
    'model_name_or_path': model_name,
    'train_file': '/opt/ml/input/data/train/train.json',
    'validation_file': '/opt/ml/input/data/val/val.json',
    'do_train': True,
    'do_eval': False,
    'fp16': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

In [ ]:
huggingface_estimator_qa = HuggingFace(entry_point='run_qa.py',
                                       source_dir='./scripts',
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       volume_size=volume_size,
                                       role=role,
                                       transformers_version='4.10',
                                       pytorch_version='1.9',
                                       py_version='py38',
                                       hyperparameters=hyperparameters_qa,
                                       disable_profiler=True,
                                      )

In [ ]:
huggingface_estimator_qa.fit({'train': f's3://{sagemaker_session_bucket}/{s3_prefix_qa}/', 'val': f's3://{sagemaker_session_bucket}/{s3_prefix_qa}/'}, wait=False)

### Model Deployment

In [ ]:
predictor_qa = huggingface_estimator_qa.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    wait=False,
    endpoint_name="turkish-qa-endpoint"
)

### Model Testing

In [ ]:
# This is only required to create a predictor from an already deployed model
predictor_qa = HuggingFacePredictor('turkish-qa-endpoint')

In [ ]:
#Question: "When did he start a vagabond life?"
#Predicted answer: "On his father's death"

data = {
"inputs": {
    "question": "Ne zaman avare bir hayata başladı?",
    "context": """ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."""
    }
}
predictor_qa.predict(data)['answer']

In [ ]:
#Question: "When did Einstein return to Germany?"
#Predicted answer: "1914"

data = {
"inputs": {
    "question": "Ne zaman Almanya’ya döndü?",
    "context": """1908’de artık oldukça tanınmış, büyük bir bilim adamı olarak tanınıyordu ve Bern \
Üniversitesinde öğretmen olarak atanmıştı. Sonraki sene patent ofisindeki işinden ve öğretmenlikten \
ayrıldı ve Zürih Üniversitesinde fizik doçentliğine başladı. 1911 yılında Prag’da Karl-Ferdinand \
Üniversitesinde profesörlük unvanı aldı. 1914 yılında Almanya’ya döndü, Kaiser Willhelm Fizik \
Enstitüsü’nde yönetici, Berlin Humboldt Üniversitesinde profesör oldu. Bu işlerindeki \
sözleşmelerinde öğretmenlik görevlerini oldukça azaltan maddeler vardı."""
    }
}
predictor_qa.predict(data)['answer']